In [1]:
# This is Main function.
# Extracting streaming data from Twitter, pre-processing, and loading into MySQL
import credentials # Import api/access_token keys from credentials.py
import settings # Import related setting constants from settings.py 

import re
import tweepy
import mysql.connector
import pandas as pd
from textblob import TextBlob


# Override tweepy.StreamListener to add logic to on_status
class MyStreamListener(tweepy.Stream):
    '''
    Tweets are known as “status updates”. So the Status class in tweepy has properties describing the tweet.
    https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object.html
    '''
    
    def on_status(self, status):
        '''
        Extract info from tweets
        '''
        
        if status.retweeted:
            # Avoid retweeted info, and only original tweets will be received
            return True
        # Extract attributes from each tweet
        id_str = status.id_str
        created_at = status.created_at
        text = deEmojify(status.text)    # Pre-processing the text  
        sentiment = TextBlob(text).sentiment
        polarity = sentiment.polarity
        subjectivity = sentiment.subjectivity
        
        user_created_at = status.user.created_at
        user_location = deEmojify(status.user.location)
        user_description = deEmojify(status.user.description)
        user_followers_count =status.user.followers_count
        
        longitude = None
        latitude = None
        if status.coordinates:
            longitude = status.coordinates['coordinates'][0]
            latitude = status.coordinates['coordinates'][1]
            
        retweet_count = status.retweet_count
        favorite_count = status.favorite_count
        status.user
        print(status.text)
        print("Long: {}, Lati: {}".format(longitude, latitude))
        
        # Store all data in MySQL
        if mydb.is_connected():
            mycursor = mydb.cursor()
            sql = "INSERT INTO {} (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location, user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)".format(settings.TABLE_NAME)
            val = (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location, \
                user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count)
            mycursor.execute(sql, val)
            mydb.commit()
            mycursor.close()
    
    
    def on_error(self, status_code):
        '''
        Since Twitter API has rate limits, stop srcraping data as it exceed to the thresold.
        '''
        if status_code == 420:
            # return False to disconnect the stream
            return False

In [2]:
settings.TABLE_ATTRIBUTES

'id_str VARCHAR(255), created_at DATETIME, text VARCHAR(255),             polarity INT, subjectivity INT,              user_created_at VARCHAR(255), user_location VARCHAR(255),             user_description VARCHAR(255), user_followers_count INT, longitude DOUBLE, latitude DOUBLE,             retweet_count INT, favorite_count INT'

In [3]:
def clean_tweet(self, text): 
    ''' 
    Use sumple regex statemnents to clean tweet text by removing links and special characters
    '''
     # removing unwanted digits ,special chracters from the text

    #url_regex = "http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"
    #handle_regex= "^@?(\w){1,15}$"
    text = text.lower()
    text= ' '.join(re.sub('\w+\s@\w+:+', ' ', text).split())
    text= ' '.join(re.sub('@\w+:+', ' ', text).split())
    #text= ' '.join(re.sub("(@[A-Za-z0-9]+)", " ", text).split()) #tags
    text= ' '.join(re.sub("^@?(\w){1,15}$", " ", text).split())
    text= ' '.join(re.sub("(\w+:\/\/\S+)", " ", text).split())   #Links
    text= ' '.join(re.sub("http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"," ", text).split()) 
    text= re.sub(r'<[^<]+?>', '', text)  #xml format
    text= text.replace('\n', ' ') # newline
    text= re.sub(r'\s+', ' ', text) # spaces
    text= re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text= re.sub('\w*\d\w*', ' ', text)
    text= ' '.join(re.sub(r'www\S+', '',text).split())
    text= ' '.join(re.sub("\s+", " ",text).split()) #Extrem white Space
    text= ' '.join(re.sub("[^-9A-Za-z ]", "" ,text).split()) #digits 
    text= ' '.join(re.sub('-', ' ', text).split()) 
    text= ' '.join(re.sub('_', ' ', text).split()) #underscore 
    RE_EMOJI = re.compile('[\U00010000-\U0010ffff]', flags=re.UNICODE)
    text = RE_EMOJI.sub(r'', text)
    return text
    
   # return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) |(\w+:\/\/\S+)", " ", tweet).split()) 
def deEmojify(text):
    '''
    Strip all non-ASCII characters to remove emoji characters
    '''
    if text:
        return text.encode('ascii', 'ignore').decode('ascii')
    else:
        return None

In [4]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="xxxxxx",
    database="tweetdb",
    charset = 'utf8'
)
if mydb.is_connected():
    '''
    Check if this table exits. If not, then create a new one.
    '''
    mycursor = mydb.cursor()
    mycursor.execute("""
        SELECT COUNT(*)
        FROM information_schema.tables
        WHERE table_name = '{0}'
        """.format(settings.TABLE_NAME))
    if mycursor.fetchone()[0] != 1:
        mycursor.execute("CREATE TABLE {} ({})".format(settings.TABLE_NAME, settings.TABLE_ATTRIBUTES))
        mydb.commit()
    mycursor.close()
    

In [5]:
settings.TABLE_ATTRIBUTES

'id_str VARCHAR(255), created_at DATETIME, text VARCHAR(255),             polarity INT, subjectivity INT,              user_created_at VARCHAR(255), user_location VARCHAR(255),             user_description VARCHAR(255), user_followers_count INT, longitude DOUBLE, latitude DOUBLE,             retweet_count INT, favorite_count INT'

In [6]:
# auth  = tweepy.OAuthHandler(credentials.CONSUMER_KEY, credentials.CONSUMER_SECRET)
# auth.set_access_token(credentials.ACCESS_TOKEN, credentials.ACCESS_TOKEN_SECRET)
# api = tweepy.API(auth)

In [7]:
myStreamListener = MyStreamListener(credentials.CONSUMER_KEY, credentials.CONSUMER_SECRET, credentials.ACCESS_TOKEN, credentials.ACCESS_TOKEN_SECRET)
#myStream = tweepy.Stream(credentials.CONSUMER_KEY, credentials.CONSUMER_SECRET, credentials.ACCESS_TOKEN, credentials.ACCESS_TOKEN_SECRET)
myStreamListener.filter(languages=["en"], track = settings.TRACK_WORDS)
mydb.close()

RT @TWICEANALYTICS: .@JYPETWICE’s ‘Merry &amp; Happy’ re-enters Apple Music South Korea Albums Top 200 at #189 🇰🇷🎄 https://t.co/I2zRNyUeC0
Long: None, Lati: None
The Nigerian Experience Podcast #TNEP

New Episode "Relationship Chronicles" is out now on various podcast listenin… https://t.co/QyAjvSkiIx
Long: None, Lati: None
RT @FNBons: The gang ⁦@RjoRecords⁩ with another one! ⁦@Rosco_Ty⁩ 🔥👐🏽. OUT NOW! https://t.co/IRsh6lzF6Q
Long: None, Lati: None
the new NYPD mostly made up of former doormen, apple store employees and dudes who failed the fire fighter test
Long: None, Lati: None
Apple Music keeps playing stick talk on auto play I’m sick of the song now
Long: None, Lati: None
This material uses a physics trick to keep roofs cool in summer and warm in winter - Popular Science https://t.co/5Y7g7D4jVr
Long: None, Lati: None
RT @JonilarNet: Follow the link to listen to #Yekoema by @KeenyIce . Crazy #Amapiano from Ghana. 
✅@audiomack  - https://t.co/Ie7gjGMGRn

✅…
Long: None, Lati: None
Was

RT @oliviazao: Apple Pay is not fake money
Long: None, Lati: None
Warre Buffett Says You Can Spot a Great Leader by Simply Looking for This 1 Trait - Inc. https://t.co/Y9L1qK0A1f
Long: None, Lati: None
The development makes good on the technology company’s pledge to boost user privacy.
Read:https://t.co/qE1N0BuxDK… https://t.co/PYvI8PNlpY
Long: None, Lati: None
I ain’t taking your shit todayyy  https://t.co/EgY4BId1a8
Long: None, Lati: None
and y’all can stay mad cuz she ate https://t.co/jmJyxhpetn
Long: None, Lati: None
RT @hongbin4council: “Swimming upstream in a tide of money” is the effort to preserve forest, so far yielded little to reverse the global t…
Long: None, Lati: None
RT @ZuvanDhaduveria: Rose apple.  #addumeedhoo https://t.co/HfkjGnItny
Long: None, Lati: None
RT @LGwenn: Apple bottom Jorts 
Butter on his fur

Got the whole company looking at purr

He hit the door

Next thing you know

Closet was…
Long: None, Lati: None
RT @Thefrogkilller: Ameno Amapiano is #23 on UK Appl

RT @mkyl_mrqz: It's the adam's apple for me✨
Long: None, Lati: None
RT @StarboyEurope: UK Apple Music songs chart:
94 B. D’OR - Burna X Wizkid (+55)
New peak! 
Keep streaming!

https://t.co/MswLObJmfO https:…
Long: None, Lati: None
Whenever I read this kind of exploit writeups I’m kind of sad that the core vulnerability of reads out of bound jus… https://t.co/zlQHTRMqPU
Long: None, Lati: None
Thank you Angelo🙏🏾🙏🏾 https://t.co/poEkPw9uc9
Long: None, Lati: None
$AAPL IV Index mean is at 34 compared to 52-week range of 19 to 49 with 30 strikes trading more than 1K contracts a… https://t.co/7PYRWEBdoQ
Long: None, Lati: None
RT @Mars_Labs: The Mars Mining App Just Launched!!!

Please click  Like and retweet this post, also tag 3 friends and comment with your App…
Long: None, Lati: None
RT @AppleTV: An impossible decision awaits. Starring Mahershala Ali, Naomie Harris, with Awkwafina and Glenn Close, #SwanSong now in cinema…
Long: None, Lati: None
Orange juice don’t even quench your thirst f

You can cancel the car wash, ima pay for it https://t.co/EdAKsrTe6t
Long: None, Lati: None
this @Meta ad has everything Apple, Coinbase, Binance, Metaverse, ICO 

please don't spend your money here, they ev… https://t.co/rEUbyWwHJe
Long: None, Lati: None
sent song 21 days (+apple music) #จะทักเดมคนที่มารีทวิต
Long: None, Lati: None
RT @hourIyng: #SUGA is the first &amp; only korean soloist to enter the top 100 of US apple music! https://t.co/AJnxgHn0B0
Long: None, Lati: None
RT @kitchens_sync: If you thought this was cool, we're hiring too!
https://t.co/Nr1WMAizNj
Long: None, Lati: None
RT @BitcoinOficial: Download on the App Store: https://t.co/kfiFi2WyUB
Get it on Google Play: https://t.co/kbCgERBq15 https://t.co/7vHQtGGd…
Long: None, Lati: None
Follow the link to listen to #Yekoema by @KeenyIce . Crazy #Amapiano from Ghana. 
✅@audiomack  -… https://t.co/oXAmG9XSga
Long: None, Lati: None
First listen favorite

https://t.co/MVtMi7h0y1
Long: None, Lati: None
RT @reedgalen: It's time to

RT @CeePresh1: Apple music users watching everyone talk about their Spotify wrapped https://t.co/h8OxqBSg1A
Long: None, Lati: None
RT @CeePresh1: Apple music users watching everyone talk about their Spotify wrapped https://t.co/h8OxqBSg1A
Long: None, Lati: None
RT @masterpiecepbs: We asked and the new men of #SanditonPBS answered! Here's a sneak peek at this Sunday's final episode in our special Sa…
Long: None, Lati: None
jungkook apple pie fancam edit lizzy mcAlpine https://t.co/x1WLJmYgQ4
Long: None, Lati: None
RT @AndrewSmithClub: Imagine Uber Eats, Apple, or Netflix deciding they don't want you as a customer because they decided to check your soc…
Long: None, Lati: None
RT @AnthxnyH_: Apple Juice &gt;&gt;&gt;&gt;&gt;
Long: None, Lati: None
RT @iamjakestx: H-E-B not having Apple Pay is absurd
Long: None, Lati: None
RT @MusiqFro: • Burna Boy ft Wizkid “B.D’or” has entered UK Apple Music Top 100 chart.
Long: None, Lati: None
$AAPL lottos coming in 
Apple Option Alert: Fri $172.5 Calls

...

RT @skyjuice_za: We dropped a project last night🤝
#ParadiseAvenueEP 

https://t.co/NYT6mfGgCn
 https://t.co/AlpvthQewg
Long: None, Lati: None
RT @KellyAlfaroBeck: If you didn’t chuckle even a tiny bit please stop following me. You are not my people. 

https://t.co/8bW3on4m4V
Long: None, Lati: None
RT @SayNoToSino: China has all global Apple users data https://t.co/wRJjuovGpC
Long: None, Lati: None
took $AAPL 172.5 calls on the break and base over the vwap 
-9ema also crossed over 20 ema and vwap. (bullish) https://t.co/TiE6TWC6vf
Long: None, Lati: None
RT @jnkloops: “it’s called apple crumble pie 💁🏻‍♀️” https://t.co/xsiKHIYzuB
Long: None, Lati: None
RT @ITZY_WT: [31st Seoul Music Awards] 

Midzys, vote for ITZY on the SMA app, we need to close the distance between us and 15th place!

SM…
Long: None, Lati: None
RT @Mars_Labs: The Mars Mining App Just Launched!!!

Please click  Like and retweet this post, also tag 3 friends and comment with your App…
Long: None, Lati: None
@ShibinMeta Dm

bruh you're a lawyer and a hired gun for developers who have a vested interest in eliminating Apple's procompetitiv… https://t.co/ZBSI8i9q7j
Long: None, Lati: None
the intro is hard tho! https://t.co/TNsVjcZVgp
Long: None, Lati: None
@Topps Apple Pie 🥧
Long: None, Lati: None
Our New Streaming Channel IS now Streaming Via ROKU|APPLETV|AmazonfireTV| https://t.co/ud7xS0QlGE https://t.co/Ba8rt6GnMo
Long: None, Lati: None
RT @ReelitBackPod: Oh fudge! This week the guys discuss 'A Christmas Story'! They go over where this ranks in their #Christmas movies list,…
Long: None, Lati: None
RT @TWICEANALYTICS: .@JYPETWICE’s ‘Merry &amp; Happy’ re-enters Apple Music South Korea Albums Top 200 at #189 🇰🇷🎄 https://t.co/I2zRNyUeC0
Long: None, Lati: None
RT @DogemonGoApp: Dear Trainers!🐶

DogemonGo, the only 100% Free AR Crypto Game is now available on #Apple’s App &amp; #Google's Play Store!…
Long: None, Lati: None
RT @TWICEANALYTICS: .@JYPETWICE’s ‘Merry &amp; Happy’ re-enters Apple Music South Korea 

Damnnnn that roddy ricch album so trash that future an baby song. They tried to save that nigga. You know how when… https://t.co/WO4YR3UKOn
Long: None, Lati: None
I entered to win a #PlayStation5, #Apple gear, #Echelon Bike, and more in @FullSail's #12DaysofGear Giveaway. Enter… https://t.co/mW5FPrgC1g
Long: None, Lati: None
RT @DogemonGoApp: Dear Trainers!🐶

DogemonGo, the only 100% Free AR Crypto Game is now available on #Apple’s App &amp; #Google's Play Store!…
Long: None, Lati: None
RT @_Georgeobeid: Apple Is Reportedly Close to Making More of Its Chips In-House
#Apple #Tech #Technology #Computer #𝒟𝒾𝑒𝒷𝒪1 #𝒟𝒾𝑒𝒷𝒪37 
https…
Long: None, Lati: None
RT @m_ontatta20: 1 WEEK UNTIL CHRISTMAS TREE
DECEMBER 24, 2021 (FRIDAY) 6PM!🎄

donate funds for charting:
https://t.co/gfeBvnWpRg
https://t…
Long: None, Lati: None
@SquawkStreet @davidfaber @CNBC IBM replacing Apple as safe haven as valuations mattering will be the 2022 market t… https://t.co/stQs7vs5EO
Long: None, Lati: None
Anyone looking t

RT @lFZXLrSzTLKsc6k: @JP_Jinfanbase LINE ♬
Apple  🍎
Spotify ♬

🌍I-ARMY💜 Please retweet J-ARMY's reply🙏

#Yours_Jin Yours  JIN https://t.co/…
Long: None, Lati: None
RT @updatesCYJ: Yujin’s fellow member Seunghee released an OST called “You are my love and destiny” for Pumpkin Time! Please support it!!…
Long: None, Lati: None
My Apple Music has me in my feelings
Long: None, Lati: None
@jambzecdn Dm with your cashapp Apple Apple PayPal or Venmo or zelle then get all your Christmas 🎄🎄🎄🎄💵💵💵 bills paid up in just 20sec
Long: None, Lati: None
Bob Boilen's Favorite music of 2021 is all in one place including a 100 song playlist and  top 10 albums of 2021 He… https://t.co/wJBFQ1JG8I
Long: None, Lati: None
Elon Musk Wants SpaceX to Reach Mars Using Carbon Capture. Here's How It Could Work - Interesting Engineering https://t.co/wfju70vJDt
Long: None, Lati: None
RT @AppleMusic: This one's for the fans.

Listen to @SAINtJHN's "The Best Part Of Life": https://t.co/N55WvdrWXt https://t.co/2s5esT9jkX


KeyboardInterrupt: 